# Telecom Customer Churn Prediction Project

The goal of the present project is to maximize the

The Telco customer churn data contains information about a fictional telco company that provided home phone and Internet services to 7043 customers in California in Q3. It indicates which customers have left, stayed, or signed up for their service.

Information about the Dataset Here  here: https://ieee-dataport.org/documents/wafn-usec-telco-customer-churn  
I created a dictionary for the variables that can be find in the repository

In [1]:
#Due to changes in the version 1.6 of Scikit-learn an inferior version must be used for XGBClassifier to function correctly.
!pip uninstall -y scikit-learn
!pip install scikit-learn==1.5.2

Found existing installation: scikit-learn 1.5.2
Uninstalling scikit-learn-1.5.2:
  Successfully uninstalled scikit-learn-1.5.2


  Using cached scikit_learn-1.5.2-cp313-cp313-win_amd64.whl.metadata (13 kB)
Using cached scikit_learn-1.5.2-cp313-cp313-win_amd64.whl (11.0 MB)



[notice] A new release of pip is available: 24.3.1 -> 25.0.1
[notice] To update, run: python.exe -m pip install --upgrade pip


In [ ]:
# I import the libraries that will be used in this project 

import pandas as pd
import numpy as np
import matplotlib as mpl
import matplotlib.pyplot as plt
import seaborn as sns
from sklearn.model_selection import train_test_split
from scipy import stats
import optuna
from sklearn.ensemble import RandomForestClassifier
from sklearn.metrics import roc_auc_score, make_scorer
from sklearn.model_selection import cross_val_score

c:\Users\vicsa\AppData\Local\Programs\Python\Python313\Lib\site-packages\tqdm\auto.py:21: TqdmWarning: IProgress not found. Please update jupyter and ipywidgets. See https://ipywidgets.readthedocs.io/en/stable/user_install.html
  from .autonotebook import tqdm as notebook_tqdm


ImportError: cannot import name 'RandomForestClassifier' from 'xgboost' (c:\Users\vicsa\AppData\Local\Programs\Python\Python313\Lib\site-packages\xgboost\__init__.py)

In [ ]:
DIR = "C:/Users/vicsa/OneDrive/Documents/Personal/Projects/telecom-churn-prediction-project"
df = pd.read_csv("WA_telco_data.csv")
df.head()

In [ ]:
df.info()

In [ ]:
#calculos estadisticos de la base de datos
df.describe(include='all')

In [ ]:
df["MonthlyCharges"].hist(figsize=(8,5));
                                

In [ ]:
df["tenure"].hist(figsize=(8,5));

In [ ]:
df["Contract"].hist(figsize=(8,5));

In [ ]:
df['InternetService'].value_counts()

In [ ]:
df['Contract'].value_counts()

In [ ]:
df['PaymentMethod'].value_counts()

In [ ]:
df['Churn'].value_counts().plot(kind='bar')
plt.show()

# Feature Engineering

In [ ]:
# I create dummies for the boolean columns
df = pd.get_dummies(df, prefix=['gender', 'PaymentMethod', 'InternetService','Contract'], columns=['gender', 'PaymentMethod', 'InternetService','Contract'])
df.head()

In [ ]:
# I eliminate the column gender_Male for correlation with gender_Female
df = df.drop('gender_Male', axis=1)
df.head()

In [ ]:
# Nophoneserveice /NoInternetService in columns that have it
df['MultipleLines'] = df['MultipleLines'].map({'No': 0, 'No phone service': 0, 'Yes': 1})
df['OnlineSecurity'] = df['OnlineSecurity'].map({'No': 0, 'No internet service': 0, 'Yes': 1})
df['OnlineBackup'] = df['OnlineBackup'].map({'No': 0, 'No internet service': 0, 'Yes': 1})
df['DeviceProtection'] = df['DeviceProtection'].map({'No': 0, 'No internet service': 0, 'Yes': 1})
df['TechSupport'] = df['TechSupport'].map({'No': 0, 'No internet service': 0, 'Yes': 1})
df['StreamingTV'] = df['StreamingTV'].map({'No': 0, 'No internet service': 0, 'Yes': 1})
df['StreamingMovies'] = df['StreamingMovies'].map({'No': 0, 'No internet service': 0, 'Yes': 1})

df.head()

In [ ]:
df['Partner'] = df['Partner'].map({'No': 0, 'Yes': 1})
df['Dependents'] = df['Dependents'].map({'No': 0, 'Yes': 1})
df['PhoneService'] = df['PhoneService'].map({'No': 0, 'Yes': 1}) 
df['PaperlessBilling'] = df['PaperlessBilling'].map({'No': 0, 'Yes': 1}) 
df['Churn'] = df['Churn'].map({'No': 0, 'Yes': 1})

df.head()

In [ ]:
df[['TotalCharges']] = df[['TotalCharges']].apply(lambda col: pd.to_numeric(col, errors='coerce'))
df.info()

In [ ]:
df['ChargesAmount'] = df['TotalCharges']/df['MonthlyCharges']
df.info()

# Training Strategy


In [ ]:
df_training = df.select_dtypes(include=['float64', 'int64', 'int32', 'int16', 'int8', 'bool'])

primary_seed = 990653
X = df_training[df_training.columns.drop('Churn')]
y = df_training['Churn']

In [ ]:
# I Split the dataset into Train, Validation and Test.
X_trainval, X_test, y_trainval, y_test = train_test_split(
    X, y, test_size=0.25, random_state=primary_seed
)

X_train, X_val, y_train, y_val = train_test_split(
    X_trainval, y_trainval, test_size=1/3, random_state=primary_seed
)

print(len(X_train), len(X_val), len(X_test))

# Bayesian Optimization

Explain why Bayesian and why XGBoos (Handle NA values, velocity,etc)
max_depth': sp_randint(3,6),
              'learning_rate': sp_randfloat(0.01, 0.1),
              'n_estimators': sp_randint(100, 1000),
              'subsample': [0.6, 0.7, 0.8, 0.9, 1.0],
              'reg_alpha': sp_randfloat(0.0, 1.0),
              'reg_lambda': sp_randfloat(0.0, 1.0),

In [ ]:
def objective(trial):
    param = {
        'n_estimators': trial.suggest_int('n_estimators', 50, 500),
        'max_depth': trial.suggest_int('max_depth', 2, 20),
        'min_samples_split': trial.suggest_int('min_samples_split', 2, 10),
        'min_samples_leaf': trial.suggest_int('min_samples_leaf', 1, 10),
        'max_features': trial.suggest_categorical('max_features', ['sqrt', 'log2', None]),
        'bootstrap': trial.suggest_categorical('bootstrap', [True, False])
    }
    clf = RandomForestClassifier(**param, random_state=primary_seed)
    score = cross_val_score(clf, X_train, y_train, cv=3, scoring='roc_auc').mean()
    return score

study = optuna.create_study(direction='maximize', sampler=optuna.samplers.TPESampler(seed=primary_seed))
study.optimize(objective, n_trials=50)


# Final Training & Evaluation

In [ ]:
# Get the best parameters and the model 
best_params = study.best_params
clf = RandomForestClassifier(**best_params, random_state=primary_seed)
clf.fit(X_train, y_train)

In [ ]:
#Review with Validation Dataset
# Predict probabilities for the validation set
y_val_proba = clf.predict_proba(X_val)[:, 1]  # probabilities for the positive class

# Calculate ROC AUC for validation set
val_auc = roc_auc_score(y_val, y_val_proba)
print(f"Validation ROC AUC: {val_auc}")

In [ ]:
#Combine Training with Validation Data
# Combine the features
X_combined = np.concatenate([X_train, X_val], axis=0)
# Combine the labels
y_combined = np.concatenate([y_train, y_val], axis=0)

In [ ]:
#Retrain the Final Model
# Initialize with best parameters (as before)
clf_final = RandomForestClassifier(**best_params, random_state=primary_seed)

# Train on the combined training and validation dataset
clf_final.fit(X_combined, y_combined)

# Predict in Final Test

In [ ]:
# Predict probabilities for the test set
y_test_proba = clf_final.predict_proba(X_test)[:, 1]  # Probabilities for the positive class

In [ ]:
#Evaluate performance using ROC-AUC
test_auc = roc_auc_score(y_test, y_test_proba)
print(f"Test ROC AUC: {test_auc}")

# Scoring

Get an Outcome that is workable for Finance reasons or something like that

In [ ]:
# Predict churn for the test set
y_test_pred = clf_final.predict(X_test)

# Create a DataFrame with actual and predicted values
output_df = pd.DataFrame({
    'Predicted_Churn': y_test_pred,
    'Actual_Churn': y_test.values
})

In [ ]:
# Define revenue calculation function
def calculate_revenue(predicted, actual):
    if predicted == 1 and actual == 1:
        return 100000
    elif predicted == 0 and actual == 0:
        return 0
    elif predicted == 1 and actual == 0:
        return -5000 #change to 5000

# Apply revenue function to each row
output_df['Revenue_Score'] = output_df.apply(lambda row: calculate_revenue(row['Predicted_Churn'], row['Actual_Churn']), axis=1)

# Calculate total revenue
total_revenue = output_df['Revenue_Score'].sum()

# Display total revenue
print(f"Total Revenue: ${total_revenue}")

# Save to CSV file
output_df.to_csv("telco_churn_predictions_revenue.csv", index=False)